Things to consider

- The source of the data OWM or OM
- Which models to train rfr, xgb, knn, ridge by settting the True or False labels
- Which models to train by giving the Deep Learning names

# Imports

In [4]:
from shared_utilities import *

# Utilities

In [12]:
def setup_data(data_ = f'data\Sere Wind Farm_hourly_OWM.csv', train_ = False, uni = True, window_size = 24*4, step = 24, sanity_check = False, tensor_ = False):
    if uni:
        column_ = 0
    else:
        column_ = None

    dm = WeatherDataModule(data_dir=data_, 
                        window_size=window_size, column=column_,
                        batch_size=32, step_=step, 
                        normalize_=True, return_tensor=tensor_)

    dm.prepare_data()
    dm.setup('')

    if sanity_check:
        plt.plot(np.arange(window_size),dm.f_test[0], label='Input')
        if step == 1:
            plt.scatter(np.arange(window_size, window_size+step),dm.t_test[0], label='Target', s=5, c='r')
        else:
            plt.plot(np.arange(window_size, window_size+step),dm.t_test[0], label='Target', c='r')
        plt.legend()
        plt.show()

    return dm

def train_deep_models(dm, window_size, step, source, name, folder='deep_models'):
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model = build_model(hidden_size=[64, 32], out=step, input_shape_= window_size, type_=name)

    checkpoint_path = f"{folder}/keras_model_{name}_ws_{window_size}_{step}_{source}.h5"

    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')

    model.compile(optimizer='adam', loss='mse')

    model.fit(dm.f_train, dm.t_train, validation_data=(dm.f_valid, dm.t_valid), epochs=150, batch_size=32, verbose=1, callbacks=[early_stop, checkpoint])

    return model



# Setup

In [17]:
window_size = 24*4
step = 24
source = 'OWM'
save_folder = 'models_compare'
train_reg = True
train_deep = False

dm = setup_data(data_=f'data\Sere Wind Farm_hourly_{source}.csv', train_ = False, uni = True, window_size = window_size, step = step, sanity_check = False, tensor_=False)

Train: (6050, 96)
Valid: (1729, 96)
Test: (744, 96)


# Train Regression

In [18]:
if train_reg:
    train(dm= dm, folder=save_folder, train_models=True, rfr=True, xgb_=True, knn=True, ridge=True, window_size=window_size, step=step)

Train: (6050, 96)
Valid: (1729, 96)
Test: (744, 96)
Training Random Forest Regressor...


KeyboardInterrupt: 

# Train Deep Models

In [13]:
if train_deep:
    model_names = ['DNN', 'LSTM', 'GRU', 'CNN']

    for name in model_names:
        train_deep_models(dm, window_size, step, source, name, folder=save_folder)

Epoch 1/150
190/190 [==============================] - 1s 2ms/step - loss: 0.0474 - val_loss: 0.0266
Epoch 2/150
171/190 [==========================>...] - ETA: 0s - loss: 0.0253

c:\Users\23603526\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


190/190 [==============================] - 0s 1ms/step - loss: 0.0252 - val_loss: 0.0210
Epoch 3/150
190/190 [==============================] - 0s 1ms/step - loss: 0.0237 - val_loss: 0.0204
Epoch 4/150
190/190 [==============================] - 0s 1ms/step - loss: 0.0233 - val_loss: 0.0198
Epoch 5/150
190/190 [==============================] - 0s 1ms/step - loss: 0.0230 - val_loss: 0.0196
Epoch 6/150
190/190 [==============================] - 0s 2ms/step - loss: 0.0227 - val_loss: 0.0197
Epoch 7/150
190/190 [==============================] - 0s 1ms/step - loss: 0.0225 - val_loss: 0.0192
Epoch 8/150
190/190 [==============================] - 0s 1ms/step - loss: 0.0223 - val_loss: 0.0187
Epoch 9/150
190/190 [==============================] - 0s 2ms/step - loss: 0.0222 - val_loss: 0.0188
Epoch 10/150
190/190 [==============================] - 0s 2ms/step - loss: 0.0219 - val_loss: 0.0190
Epoch 11/150
190/190 [==============================] - 0s 1ms/step - loss: 0.0219 - val_loss: 0.0190
